In [1]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
from sklearn.preprocessing import StandardScaler

In [2]:
list

list

In [3]:
df = pd.read_csv('data/full_data_2011-01-01_2022-11-26.csv', sep=',')

df = df.loc[[2]].drop(columns='Unnamed: 0').T
df.columns = ['energie']
df['The_date'] = df['energie']
df


,energie,The_date
01 Jan 11 Daily peak [MW],3460.0,3460.0
02 Jan 11 Daily peak [MW],3280.0,3280.0
03 Jan 11 Daily peak [MW],3577.0,3577.0
04 Jan 11 Daily peak [MW],3318.0,3318.0
05 Jan 11 Daily peak [MW],3397.0,3397.0
...,...,...
23 Nov 22 Daily peak [MW],4266.0,4266.0
24 Nov 22 Daily peak [MW],4731.0,4731.0
25 Nov 22 Daily peak [MW],4907.0,4907.0
26 Nov 22 Daily peak [MW],4489.0,4489.0


In [4]:
df_rain = pd.read_csv('data/data_precipitation+2weeks.csv',sep=';')
df_rain

,data,Alqueva,Castelo de bode,Marateca,Caia,Maranhão- Avis,Douro- Cabreira,Douro- Vilar,Douro-Tua,Alto- Lindoso,...,Valeira,Bemposta,Carrapatelo,Pocinho,Picote,Baixo Sabor,Régua,Crestuma - Lever,Varosa,Torrão
0,01.01.2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,...,0.2,0.0,0.0,0.0,0.2,0.2,0.2,0.0,0.2,0.0
1,02.01.2018,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,2.0,...,1.1,0.0,0.0,0.0,1.1,1.1,1.1,0.0,1.1,0.0
2,03.01.2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,...,0.6,0.0,0.0,0.0,0.6,0.6,0.6,0.0,0.6,0.0
3,04.01.2018,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.5,...,0.3,0.0,0.0,0.0,0.3,0.3,0.3,0.0,0.3,0.0
4,05.01.2018,2.9,2.8,2.2,1.2,2.4,0.0,1.7,0.0,5.9,...,1.5,0.0,0.0,0.0,1.5,1.5,1.5,0.0,1.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1784,20.11.2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1785,21.11.2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1786,22.11.2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1787,23.11.2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_temp= pd.read_csv('data/temperature_data+2weeks.csv', sep=';')
df_temp

,data,Alqueva,Castelo de Bode,Marateca,Miranda Douro,Veleira,Bemposta,Pocinho,Régua,Picote,Baixo Sabor,Varosa,Foz Tua,Carrapatelo,Crestuma-Lever,Vilar,Torrão
0,01.01.2018,8.750,9.50,7.00,0.000,0.000,0.000,0.000,7.00,7.00,7.00,7.00,0.00,0.00,0.00,6.25,0.00
1,02.01.2018,9.750,11.00,8.50,0.000,0.000,0.000,0.000,9.75,9.75,9.75,9.75,0.00,0.00,0.00,8.75,0.00
2,03.01.2018,12.000,13.50,11.50,0.000,0.000,0.000,0.000,11.50,11.50,11.50,11.50,0.00,0.00,0.00,11.25,0.00
3,04.01.2018,12.000,12.50,10.75,0.000,0.000,0.000,0.000,11.00,11.00,11.00,11.00,0.00,0.00,0.00,10.5,0.00
4,05.01.2018,11.000,11.00,9.75,0.000,0.000,0.000,0.000,8.00,8.00,8.00,8.00,0.00,0.00,0.00,7.75,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1784,20.11.2022,16.000,12.25,10.25,11.125,11.125,11.125,11.125,10.75,10.75,10.75,10.75,11.50,11.50,15.26,13,11.50
1785,21.11.2022,17.125,15.00,11.50,11.175,11.175,11.175,11.175,11.25,11.25,11.25,11.25,13.00,11.50,16.26,13.1,11.50
1786,22.11.2022,17.175,16.26,13.75,13.000,13.000,13.000,13.000,12.00,12.00,12.00,12.00,13.60,12.50,17.00,13.25,12.50
1787,23.11.2022,16.600,15.26,13.75,13.750,13.750,13.750,13.750,12.25,12.25,12.25,12.25,12.75,12.25,16.26,13.5,12.25


In [6]:

dict_month = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08','Sep': '09','Oct': '10','Nov': '11','Dec': '12'}
for i in range(len(df)):
    day = "".join(re.findall("\d", df.index[i].split(" ", 1)[0]))
    month = dict_month[df.index[i].split(" ")[1]]
    year = "".join(re.findall("\d", df.index[i].split(" ", 1)[1]))
    the_date = f'{day}-{month}-{year}'
    df['The_date'].iloc[i] = the_date
df['The_date'] = pd.to_datetime(df['The_date'])

In [7]:
df = df[2557:-18]
df.reset_index(inplace = True)
df

,index,energie,The_date
0,01 Jan 18 Daily peak [MW],944.0,2018-01-01
1,02 Jan 18 Daily peak [MW],2496.0,2018-02-01
2,03 Jan 18 Daily peak [MW],2576.0,2018-03-01
3,04 Jan 18 Daily peak [MW],1972.0,2018-04-01
4,05 Jan 18 Daily peak [MW],3649.0,2018-05-01
...,...,...,...
1769,05 Nov 22 Daily peak [MW],3445.0,2022-05-11
1770,06 Nov 22 Daily peak [MW],1901.0,2022-06-11
1771,07 Nov 22 Daily peak [MW],2631.0,2022-07-11
1772,08 Nov 22 Daily peak [MW],3023.0,2022-08-11


In [8]:
df_rain['mean'] = df_rain.mean(axis=1)
df_temp['mean'] = df_temp.mean(axis=1)


df['rain'] = df_rain['mean']
df['temp'] = df_temp['mean']
df


/tmp/ipykernel_14785/1063625865.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_rain['mean'] = df_rain.mean(axis=1)
/tmp/ipykernel_14785/1063625865.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_temp['mean'] = df_temp.mean(axis=1)
/tmp/ipykernel_14785/1063625865.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rain'] = df_rain['mean']
/tmp/ipykernel_14785/1063625865.py:6: SettingWithCopyWarn

,index,energie,The_date,rain,temp
0,01 Jan 18 Daily peak [MW],944.0,2018-01-01,0.068182,3.550000
1,02 Jan 18 Daily peak [MW],2496.0,2018-02-01,0.409091,4.550000
2,03 Jan 18 Daily peak [MW],2576.0,2018-03-01,0.190909,5.533333
3,04 Jan 18 Daily peak [MW],1972.0,2018-04-01,0.109091,5.283333
4,05 Jan 18 Daily peak [MW],3649.0,2018-05-01,1.500000,4.250000
...,...,...,...,...,...
1769,05 Nov 22 Daily peak [MW],3445.0,2022-05-11,0.000000,10.700000
1770,06 Nov 22 Daily peak [MW],1901.0,2022-06-11,0.131818,11.683333
1771,07 Nov 22 Daily peak [MW],2631.0,2022-07-11,0.036364,12.400000
1772,08 Nov 22 Daily peak [MW],3023.0,2022-08-11,2.759091,13.550000
